<a href="https://colab.research.google.com/github/zigzager/SF/blob/main/generador_intervalo_v3.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- INICIO DE CELDA 1 ---
%%writefile visualizador_core.py
# --- ARCHIVO 1: visualizador_core.py ---

# --- Apartado 1: Importaciones del Corazón ---
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# --- Apartado 9: El "Corazón" del Dibujo (Función Interna) ---
def crear_grafico_interno(ruta_csv, tipo_grafico, fecha_inicio, fecha_fin, nombre_archivo_salida):
    """
    Esta es la función "tonta". No pregunta NADA.
    Solo recibe los parámetros y dibuja la imagen.
    """

    # --- Lógica del Apartado 3 (Carga de CSV) ---
    try:
        separador = '\t'
        df = pd.read_csv(
            ruta_csv,
            sep=separador,
            parse_dates={'Datetime': ['<DATE>', '<TIME>']}
        )
        df.rename(columns={
            '<OPEN>': 'Open',
            '<CLOSE>': 'Close',
            '<HIGH>': 'High',
            '<LOW>': 'Low'
        }, inplace=True)
        df.set_index('Datetime', inplace=True)
        df = df[['Open', 'High', 'Low', 'Close']]
    except Exception as e:
        print(f"Error interno (Apartado 3): {e}")
        return

    # --- Lógica del Apartado 5 (Filtrado) ---
    datos_intervalo = df.loc[fecha_inicio : fecha_fin].copy()
    if datos_intervalo.empty:
        print(f"Error interno (Apartado 5): No hay datos para {fecha_inicio}-{fecha_fin}")
        return

    # --- Lógica del Apartado 6 (Dibujo) ---
    fig, ax = plt.subplots(figsize=(6, 4), dpi=300)
    fig.patch.set_facecolor('black')
    ax.set_facecolor('black')

    numero_de_puntos = len(datos_intervalo)

    # --- ¡CAMBIO AQUÍ! ---
    # Lógica de Grosor de Línea (M1 y H1)
    # Basada en tu nuevo umbral de 480 barras (8 horas M1)
    if numero_de_puntos > 480:
        line_width = 0.7 # Nuevo valor fino
    else:
        line_width = 1   # Valor grueso

    # Lógica de Tamaño de Punto (Solo M1)
    # Mantenemos el umbral de 300 barras
    if numero_de_puntos > 300:
        marker_size_m1 = 0.3
    else:
        marker_size_m1 = 2
    # --- FIN DEL CAMBIO ---

    x_axis = range(len(datos_intervalo))
    y_open = datos_intervalo['Open'].values
    y_low = datos_intervalo['Low'].values
    y_high = datos_intervalo['High'].values
    y_close = datos_intervalo['Close'].values

    for i in x_axis:
        if y_close[i] >= y_open[i]:
            color_barra = 'lime'
        else:
            color_barra = 'red'

        if tipo_grafico == '1':
            marker_size_actual = marker_size_m1
        else: # H1
            marker_size_actual = 0.3

        ax.vlines(i, ymin=y_low[i], ymax=y_high[i], color=color_barra, linewidth=line_width)
        ax.plot(i, y_close[i], 'o', color=color_barra, markersize=marker_size_actual)

    ax.axis('off')
    ax.margins(0, 0)
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

    # --- Lógica del Apartado 7 (Guardado) ---
    plt.savefig(nombre_archivo_salida, format='png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    print(f"\n¡Éxito! Gráfico (Interno) guardado como: '{nombre_archivo_salida}'")
# --- FIN DE visualizador_core.py ---
# --- FIN DE CELDA 1 ---



Overwriting visualizador_core.py


In [2]:
# --- ARCHIVO 2: app.py (corriendo en la celda 2) ---

# --- Apartado 1: Importaciones de la App ---
from datetime import datetime
import uuid
# --- ¡IMPORTANTE! Importamos nuestra propia herramienta ---
from visualizador_core import crear_grafico_interno
# --- FIN APARTADO 1 ---


# --- Apartado 2: Definición de la Función (Interfaz de Usuario) ---
def generar_imagen_intervalo(ruta_csv, formato_fecha, tipo_grafico):
    """
    Esta función es la "Interfaz de Usuario".
    Pregunta las fechas (Apartado 4) y luego llama
    al "corazón" (crear_grafico_interno) para hacer el trabajo.
    """

    # --- APARTADO 4 (DENTRO del Apartado 2) ---
    while True:
        try:
            print(f"\nIntroduce las fechas en formato: {formato_fecha}")
            if formato_fecha == '%Y-%m-%d %H:%M':
                ejemplo_inicio = "2024-10-20 14:30"
                ejemplo_fin = "2024-10-20 16:00"
            else: # Asumimos H1
                ejemplo_inicio = "2024-10-20 14"
                ejemplo_fin = "2024-10-20 20"

            fecha_inicio_str = input(f"Introduce fecha/hora de INICIO (ej: {ejemplo_inicio}): ")
            fecha_inicio = datetime.strptime(fecha_inicio_str, formato_fecha)

            fecha_fin_str = input(f"Introduce fecha/hora de FIN (ej: {ejemplo_fin}):   ")
            fecha_fin = datetime.strptime(fecha_fin_str, formato_fecha)

            if fecha_fin <= fecha_inicio:
                print("Error: La fecha de fin debe ser posterior a la fecha de inicio.")
                continue
            break
        except ValueError:
            print(f"Error: Formato de fecha incorrecto. Debe ser: {formato_fecha}")
    # --- FIN APARTADO 4 ---

    # --- APARTADO 7 (Solo la parte del nombre de archivo) ---
    base_nombre = f"{fecha_inicio.strftime('%Y%m%d%H%M')}_{fecha_fin.strftime('%Y%m%d%H%M')}"
    xyz_aleatorio = uuid.uuid4().hex[:6]
    nombre_archivo_final = f"{base_nombre}_{xyz_aleatorio}.png"

    # --- ¡LLAMADA AL CORAZÓN! ---
    # La Interfaz (app.py) le pasa el trabajo al Corazón (visualizador_core.py)
    crear_grafico_interno(ruta_csv, tipo_grafico, fecha_inicio, fecha_fin, nombre_archivo_final)
# --- FIN APARTADO 2 ---


# --- APARTADO 8: Bloque de Ejecución Principal ---
if __name__ == "__main__":

    # Define las rutas de TUS DOS archivos (súbelos a Colab)
    ruta_m1 = "XAUUSD_M1_202301110812_202511142349.csv"
    ruta_h1 = "XAUUSD_H1_200406110700_202511142300.csv"

    # Bucle principal del Menú
    while True:
        print("\n--- MENÚ PRINCIPAL ---")
        print("¿Qué datos quieres visualizar?")
        print("1: Gráfica de 1 Minuto (M1 - Vela O/C)")
        print("2: Gráfica de 1 Hora (H1 - Vela O/C)")
        print("q: Salir del programa")

        opcion = input("Elige una opción (1, 2, q): ")

        if opcion == '1':
            ruta_a_usar = ruta_m1
            formato_a_usar = '%Y-%m-%d %H:%M'
            tipo_grafico_a_usar = '1'

        elif opcion == '2':
            ruta_a_usar = ruta_h1
            formato_a_usar = '%Y-%m-%d %H'
            tipo_grafico_a_usar = '2'

        elif opcion == 'q' or opcion == 'Q':
            break

        else:
            print("Opción no válida. Por favor, elige 1, 2 o q.")
            continue

        generar_imagen_intervalo(ruta_a_usar, formato_a_usar, tipo_grafico_a_usar)

        while True:
            continuar = input("\n¿Quieres generar otra imagen de esta temporalidad? (s/n): ").lower()
            if continuar == 's':
                generar_imagen_intervalo(ruta_a_usar, formato_a_usar, tipo_grafico_a_usar)
            else:
                break

    print("Programa terminado.")
# --- FIN DE app.py ---


--- MENÚ PRINCIPAL ---
¿Qué datos quieres visualizar?
1: Gráfica de 1 Minuto (M1 - Vela O/C)
2: Gráfica de 1 Hora (H1 - Vela O/C)
q: Salir del programa
Elige una opción (1, 2, q): 1

Introduce las fechas en formato: %Y-%m-%d %H:%M
Introduce fecha/hora de INICIO (ej: 2024-10-20 14:30): 2025-10-24 12:00
Introduce fecha/hora de FIN (ej: 2024-10-20 16:00):   2025-10-24 20:00


/content/visualizador_core.py:18: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(



¡Éxito! Gráfico (Interno) guardado como: '202510241200_202510242000_9f8b53.png'

¿Quieres generar otra imagen de esta temporalidad? (s/n): n

--- MENÚ PRINCIPAL ---
¿Qué datos quieres visualizar?
1: Gráfica de 1 Minuto (M1 - Vela O/C)
2: Gráfica de 1 Hora (H1 - Vela O/C)
q: Salir del programa
Elige una opción (1, 2, q): q
Programa terminado.


In [3]:
# --- SCRIPT: level_scanner.py ---
# (Este script es independiente y no usa el visualizador_core.py)

import pandas as pd
from datetime import datetime
from scipy.signal import find_peaks # Usaremos esta librería para la Fase 1

# --- Fase 1: Encontrar Pivotes ---
def fase1_encontrar_pivotes(datos, sensibilidad_n):
    """
    Encuentra picos (Highs) y valles (Lows) en los datos.
    'sensibilidad_n' es el número de barras a cada lado para definir un "tope".
    """
    # Para picos (Highs):
    # 'distance' es el número mínimo de barras entre picos (nuestra 'N')
    indices_picos, _ = find_peaks(datos['High'], distance=sensibilidad_n)
    pivotes_high = datos.iloc[indices_picos]['High']

    # Para valles (Lows):
    # Invertimos los datos (multiplicamos por -1) para que find_peaks busque "valles"
    indices_valles, _ = find_peaks(-datos['Low'], distance=sensibilidad_n)
    pivotes_low = datos.iloc[indices_valles]['Low']

    return pivotes_high, pivotes_low

# --- Fase 2: Agrupar Pivotes en Zonas ---
def fase2_agrupar_zonas(pivotes, margen):
    """
    Agrupa una lista de precios de pivotes en "zonas"
    si están dentro del 'margen' de distancia.
    """
    if pivotes.empty:
        return []

    # Ordenamos todos los pivotes de menor a mayor
    pivotes_ordenados = pivotes.sort_values()

    zonas = []

    while not pivotes_ordenados.empty:
        # Empezamos una nueva zona
        # El primer pivote de la lista define el inicio de la zona
        base_pivote = pivotes_ordenados.iloc[0]

        # El rango de la zona es [base_pivote, base_pivote + margen]
        limite_zona = base_pivote + margen

        # Encontramos todos los pivotes que caen en esta zona
        pivotes_en_zona = pivotes_ordenados[pivotes_ordenados <= limite_zona]

        if not pivotes_en_zona.empty:
            # Calculamos el precio medio de la zona y el num de hits
            precio_medio_zona = pivotes_en_zona.mean()
            hits = len(pivotes_en_zona)
            zonas.append((precio_medio_zona, hits))

            # Eliminamos los pivotes que ya hemos agrupado
            pivotes_ordenados = pivotes_ordenados[pivotes_ordenados > limite_zona]
        else:
            # Si algo falla, solo eliminamos el primero
            pivotes_ordenados = pivotes_ordenados.iloc[1:]

    return zonas

# --- Fase 3: Seleccionar las Mejores Zonas ---
def fase3_filtrar_mejores(zonas, num_zonas):
    """Ordena las zonas por 'hits' y devuelve las mejores 'num_zonas'."""
    # Ordenamos por 'hits' (el segundo item, índice 1) de mayor a menor
    zonas.sort(key=lambda x: x[1], reverse=True)
    return zonas[:num_zonas]

# --- Función de Interfaz de Usuario (solo pedir fechas) ---
def pedir_fechas():
    formato_fecha = '%Y-%m-%d %H:%M' # Formato M1
    while True:
        try:
            print(f"\nIntroduce las fechas en formato: {formato_fecha}")
            fecha_inicio_str = input(f"Introduce fecha/hora de INICIO (ej: 2024-10-20 14:30): ")
            fecha_inicio = datetime.strptime(fecha_inicio_str, formato_fecha)

            fecha_fin_str = input(f"Introduce fecha/hora de FIN (ej: 2024-10-20 16:00):   ")
            fecha_fin = datetime.strptime(fecha_fin_str, formato_fecha)

            if fecha_fin <= fecha_inicio:
                print("Error: La fecha de fin debe ser posterior a la fecha de inicio.")
                continue
            return fecha_inicio, fecha_fin
        except ValueError:
            print(f"Error: Formato de fecha incorrecto. Debe ser: {formato_fecha}")

# --- Apartado Principal de Ejecución ---
if __name__ == "__main__":
    print("--- Iniciando Scanner de Niveles (Versión Standalone) ---")

    # --- Parámetros del Algoritmo (Puedes ajustarlos) ---
    RUTA_CSV_M1 = "XAUUSD_M1_202301110812_202511142349.csv"
    SENSIBILIDAD_N = 10  # N=10 -> mira 10 barras antes y 10 después para definir un "tope"
    MARGEN_ZONA = 1.0    # 1.0 -> Agrupa pivotes que estén a 1.0 punto de distancia
    NUM_NIVELES = 2      # El usuario pidió las 2 líneas más importantes

    # 1. Cargar datos (una sola vez al inicio)
    try:
        print(f"Cargando {RUTA_CSV_M1}...")
        separador = '\t'
        df_full = pd.read_csv(
            RUTA_CSV_M1,
            sep=separador,
            parse_dates={'Datetime': ['<DATE>', '<TIME>']}
        )
        # Solo necesitamos estas columnas para este algoritmo
        df_full.rename(columns={
            '<HIGH>': 'High',
            '<LOW>': 'Low'
        }, inplace=True)
        df_full.set_index('Datetime', inplace=True)
        df_full = df_full[['High', 'Low']]
        print("Datos cargados con éxito.")
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {RUTA_CSV_M1}.")
        print("Por favor, asegúrate de que el archivo está en la misma carpeta.")
        exit() # Salimos si no hay CSV
    except Exception as e:
        print(f"Error al cargar el CSV: {e}")
        exit() # Salimos si hay error

    # 2. Bucle principal del programa
    while True:
        # Pedir rango al usuario
        fecha_inicio, fecha_fin = pedir_fechas()

        # Filtrar el dataframe
        datos_intervalo = df_full.loc[fecha_inicio : fecha_fin].copy()
        if datos_intervalo.empty:
            print("No se encontraron datos para ese intervalo.")
            continue

        # --- Ejecutar Algoritmo ---
        print(f"\nAnalizando {len(datos_intervalo)} barras...")

        # Fase 1: Encontrar pivotes
        pivotes_high, pivotes_low = fase1_encontrar_pivotes(datos_intervalo, SENSIBILIDAD_N)
        # Juntamos picos y valles en una sola lista
        all_pivots = pd.concat([pivotes_high, pivotes_low])
        print(f"Fase 1: Encontrados {len(all_pivots)} pivotes (topes y suelos).")

        # Fase 2: Agrupar en Zonas
        zonas_con_hits = fase2_agrupar_zonas(all_pivots, MARGEN_ZONA)
        print(f"Fase 2: Agrupados en {len(zonas_con_hits)} zonas distintas.")

        # Fase 3: Seleccionar las mejores
        mejores_zonas = fase3_filtrar_mejores(zonas_con_hits, NUM_NIVELES)
        print("--- ¡Análisis Completo! ---")

        # 4. Mostrar Resultados por pantalla
        if not mejores_zonas:
            print("No se detectaron zonas de rebote significativas.")
        else:
            print(f"Los {NUM_NIVELES} niveles más importantes son:")
            for i, (precio, hits) in enumerate(mejores_zonas):
                # Usamos :.2f para mostrar el precio con 2 decimales
                print(f"  Nivel {i+1}: Precio (promedio) {precio:.2f} con {hits} hits/rebotes.")

        # 5. Preguntar si quiere continuar
        continuar = input("\n¿Quieres analizar otro intervalo? (s/n): ").lower()
        if continuar != 's':
            break

    print("Scanner terminado.")

--- Iniciando Scanner de Niveles (Versión Standalone) ---
Cargando XAUUSD_M1_202301110812_202511142349.csv...


/tmp/ipython-input-943897803.py:105: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df_full = pd.read_csv(


Datos cargados con éxito.

Introduce las fechas en formato: %Y-%m-%d %H:%M
Introduce fecha/hora de INICIO (ej: 2024-10-20 14:30): 2025-11-07 11:30
Introduce fecha/hora de FIN (ej: 2024-10-20 16:00):   2025-11-07 23:00

Analizando 691 barras...
Fase 1: Encontrados 102 pivotes (topes y suelos).
Fase 2: Agrupados en 24 zonas distintas.
--- ¡Análisis Completo! ---
Los 2 niveles más importantes son:
  Nivel 1: Precio (promedio) 4003.44 con 10 hits/rebotes.
  Nivel 2: Precio (promedio) 3999.81 con 9 hits/rebotes.

¿Quieres analizar otro intervalo? (s/n): s

Introduce las fechas en formato: %Y-%m-%d %H:%M
Introduce fecha/hora de INICIO (ej: 2024-10-20 14:30): 2025-11-10 01:30
Introduce fecha/hora de FIN (ej: 2024-10-20 16:00):   2025-11-10 12:30

Analizando 661 barras...
Fase 1: Encontrados 94 pivotes (topes y suelos).
Fase 2: Agrupados en 39 zonas distintas.
--- ¡Análisis Completo! ---
Los 2 niveles más importantes son:
  Nivel 1: Precio (promedio) 4078.77 con 6 hits/rebotes.
  Nivel 2: Prec